# KNNRecommender

Fit KNNRecommender and save the resulting Meta-Table and Model

In [ ]:
from qtfrecommender import KNNRecommender

recommender = KNNRecommender(supervised=True, load_default=False, n_neighbors=1)

recommender.fit(full_set_path="./data/full_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
# # recommender.fit(full_set_path="./data/train_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
recommender.save_meta_table("./qtfrecommender/methods/data/knn_meta_table.h5")
recommender.persist_model("./qtfrecommender/methods/data/KNNRecommender.joblib")

recommender = KNNRecommender(supervised=True, load_default=False, n_neighbors=3)
recommender.fit(full_set_path="./data/full_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
recommender.persist_model("./qtfrecommender/methods/data/KNNRecommender_n_eq_3.joblib")

recommender = KNNRecommender(supervised=True, load_default=False, n_neighbors=5)
recommender.fit(full_set_path="./data/full_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
recommender.persist_model("./qtfrecommender/methods/data/KNNRecommender_n_eq_5.joblib")

recommender = KNNRecommender(supervised=True, load_default=False, n_neighbors=7)
recommender.fit(full_set_path="./data/full_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
recommender.persist_model("./qtfrecommender/methods/data/KNNRecommender_n_eq_7.joblib")

recommender = KNNRecommender(supervised=True, load_default=False, n_neighbors=9)
recommender.fit(full_set_path="./data/full_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
recommender.persist_model("./qtfrecommender/methods/data/KNNRecommender_n_eq_9.joblib")

recommender = KNNRecommender(supervised=True, load_default=False, n_neighbors=11)
recommender.fit(full_set_path="./data/full_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
recommender.persist_model("./qtfrecommender/methods/data/KNNRecommender_n_eq_11.joblib")

Using KNNRecommender

In [ ]:
# from qtfrecommender import KNNRecommender
# import pandas as pd

# # When a KNNRecommender model is instantiated, it automatically loads
# # the fitted model persisted on the previous step (with _load=True by default).
# recommender = KNNRecommender()

# df = pd.read_csv("./data/datasets/winetype.csv")
# X = df.drop(columns=["class"]).values
# y = df["class"].values

# # Returns two tuples: (quantifiers ranking), (quantifiers weight by their ARR).
# # First quantifier is best, second quantifier is second best, and so on.
# ranking, weights = recommender.recommend(X, y)

# print(f"Top 1: {ranking[0]} with Weight: {weights[0]}")
# print(f"Top 2: {ranking[1]} with Weight: {weights[1]}")
# print(f"Top 3: {ranking[2]} with Weight: {weights[2]}")
# print(f"Full ranking: {ranking}")
# print(f"Weights: {weights}")

# RegressionRecommender

Fit RegressionRecommender and save the resulting Meta-Table and Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

from qtfrecommender import RegressionRecommender


recommender = RegressionRecommender(supervised=True, load_default=False, model=RandomForestRegressor(n_jobs=-1))
recommender.fit(full_set_path="./data/full_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
# recommender.fit(full_set_path="./data/train_set/", train_set_path="./data/train_set/", test_set_path="./data/test_set/")
recommender.save_meta_table("./qtfrecommender/methods/data/regression_meta_table.h5")
recommender.persist_model("./qtfrecommender/methods/data/RegressionRecommender.joblib")


recommender = RegressionRecommender(supervised=True, load_default=False, model=XGBRegressor(n_jobs=-1))
recommender.persist_model("./qtfrecommender/methods/data/RegressionRecommenderXGBOOST.joblib")


recommender = RegressionRecommender(supervised=True, load_default=False, model=SVR())
recommender.persist_model("./qtfrecommender/methods/data/RegressionRecommenderSVR.joblib")

Using RegressionRecommender

In [ ]:
# from qtfrecommender import RegressionRecommender
# import pandas as pd

# # When a RegressionRecommender model is instantiated, it automatically loads
# # the fitted model persisted on the previous step (with _load=True by default).
# recommender = RegressionRecommender()

# df = pd.read_csv("./data/datasets/winetype.csv")
# X = df.drop(columns=["class"]).values
# y = df["class"].values

# # Returns two tuples: (quantifiers ranking), (quantifiers weight by their MAE).
# # First quantifier is best, second quantifier is second best, and so on.
# ranking, weights = recommender.recommend(X, y)

# print(f"Top 1: {ranking[0]} with Weight: {weights[0]}")
# print(f"Top 2: {ranking[1]} with Weight: {weights[1]}")
# print(f"Top 3: {ranking[2]} with Weight: {weights[2]}")
# print(f"Full ranking: {ranking}")
# print(f"Sum of weights: {sum(weights)}")

# EnsembleQuantifier

You can use the resulting ranking from a recommender and feed it to the EnsembleQuantifier.

For example, we can recommend a ranking of quantifiers for a dataset and then use an ensemble of the
Top-k quantifiers to get the positive prevalence of the dataset.

In [ ]:
# from qtfrecommender import RegressionRecommender
# from ensemble_quantifier import EnsembleQuantifier
# from sklearn.model_selection import train_test_split
# import pandas as pd
# import numpy as np

# # Let's load a dataset
# df = pd.read_csv("./data/datasets/winetype.csv")
# X = df.drop(columns=["class"]).values
# y = df["class"].values

# # Instantiate a RegressionRecommender model
# recommender = RegressionRecommender()

# # We can use the recommender to get a ranking of quantifiers
# ranking, weights = recommender.recommend(X, y)

# # Let's instantiate an EnsembleQuantifier model
# # and pass the ranking and the Top-k quantifiers to it.
# # With k = 3, for example.
# # Now, when we need to get predictions, an ensemble of the Top-3 will be used.
# ensemble_qtf = EnsembleQuantifier(ranking=ranking)

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Fit the ensemble model
# ensemble_qtf.fit(X_train, y_train)

# # Predict the positive prevalence of the test set
# pred_prev = ensemble_qtf.predict(X_test)

# # Compare the prediction with the truth
# true_prev = np.sum(y_test) / len(y_test)

# print(f"{ensemble_qtf}")
# print(f"True prevalence: {true_prev}")
# print(f"Predicted prevalence: {pred_prev}")

# # We can change the method of the EnsembleQuantifier.
# # Avaliable methods are the following:
# # - "median": returns the median of the predictions
# # - "weighted": returns the weighted average of the predictions
# #
# # Default is "median".

# # Let's use the weighted method and pass the weights to the ensemble model.
# ensemble_qtf.method = "weighted"
# ensemble_qtf.weights = weights

# # Model is already fitted, so we can predict again
# pred_prev = ensemble_qtf.predict(X_test)

# print(f"\n{ensemble_qtf}")
# print(f"True prevalence: {true_prev}")
# print(f"Predicted prevalence: {pred_prev}")

# # We can also recommend the top- with k = 3 and pass it to the ensemble model.
# ranking, weights = recommender.recommend(X, y, k=3)

# ensemble_qtf.ranking = ranking
# ensemble_qtf.weights = weights

# # Model is fitted, so we can just predict
# pred_prev = ensemble_qtf.predict(X_test)

# print(f"\n{ensemble_qtf}")
# print(f"True prevalence: {true_prev}")
# print(f"Predicted prevalence: {pred_prev}")

# Evaluation

Evaluate RegressionRecommender with Leave-One-Out

In [ ]:
# from qtfrecommender import RegressionRecommender

# # There is a built-in method to evaluate the RegressionRecommender
# # using the leave-one-out strategy. It excludes one instance from the
# # meta-table and uses the remaining instances to fit the model. Then,
# # it predicts the ranking of the excluded instance and appends the
# # predicted ranking alongside the true ranking to the recommender evaluation table.
# recommender = RegressionRecommender()
# recommender.load_model()
# reg_recommender_eval, reg_quantifiers_eval = recommender.leave_one_out_evaluation("./plot_data/reg_recommender_evaluation_table.csv",
#                                                                                   "./plot_data/reg_quantifiers_evaluation_table.csv")

Evaluate KNNRecommender with Leave-One-Out

In [ ]:
# from qtfrecommender import KNNRecommender

# # There is a built-in method to evaluate the KNNRecommender
# # using the leave-one-out strategy. It excludes one instance from the
# # meta-table and uses the remaining instances to fit the model. Then,
# # it predicts the ranking of the excluded instance and appends the
# # predicted ranking alongside the true ranking to the recommender evaluation table.
# recommender = KNNRecommender()
# recommender.load_model()
# knn_recommender_eval, knn_quantifiers_eval = recommender.leave_one_out_evaluation("./plot_data/knn_recommender_evaluation_table.csv",
#                                                                                   "./plot_data/knn_quantifiers_evaluation_table.csv")

We can evaluate the EnsembleQuantifier with APP.

APP is very costly, so we don't exactly run it again. Since our model is an ensemble, we reuse the recommender's and the quantifiers evaluation table generated by the recommender. (It will "run" the APP on the same datasets used to train the recommender).

In [ ]:
# from ensemble_quantifier import EnsembleQuantifier

# # We need the recommender evaluation table and the quantifiers evaluation table
# # generated with the leave_one_out_evaluation method from the RegressionRecommender.
# ensemble_qtf = EnsembleQuantifier()
# ensemble_qtf.evaluation("regression", reg_recommender_eval, reg_quantifiers_eval, "./plot_data/reg_ensemble_quantifier_evaluation_table.csv")
# ensemble_qtf.evaluation("knn", knn_recommender_eval, knn_quantifiers_eval, "./plot_data/knn_ensemble_quantifier_evaluation_table.csv")